In [ ]:
import os
import json

folders = os.listdir()

all_data = []

def extract_sanction_data(folder):
  all_data = []
  for file in os.listdir(folder):
    if "." in file:
      continue
    current_state = []
    for data_file in os.listdir(os.path.join(folder, file)):
      with open(os.path.join(folder, file, data_file), 'r') as f:
            data = json.load(f)
            for d in data:
              d["folder_saciones"] = folder
              d["folder_estado"] = file
              d["file_data"] = data_file
              current_state.append(d)
      all_data = all_data + current_state
  return all_data



base_folder = "sanciones"
tipos = os.listdir(base_folder)
sanciones = {}
folder_priv = os.path.join(base_folder, "s3p")
sanciones["sanciones_privadas"] = extract_sanction_data(folder_priv)
folder_serv = os.path.join(base_folder, "s3s")
sanciones["sanciones_servidores"] = extract_sanction_data(folder_serv)




In [ ]:
len(sanciones['sanciones_privadas'])

12805

In [ ]:
len(sanciones['sanciones_servidores'])

36948

In [ ]:
def extract_private(objeto):
    # Extracción del ID del caso
    id_caso = objeto.get('id', 'No disponible')

    # Extracción del nombre del sancionado
    nombre_sancionado = objeto.get('particularSancionado', {}).get('nombreRazonSocial', 'No disponible')
    state_caso = objeto.get('folder_estado', 'No Disponible')

    # Intentar extraer nombres de empresas relacionadas (este campo no se encuentra en el ejemplo proporcionado)
    empresas_relacionadas = []
    if 'empresas' in objeto:
        for empresa in objeto['empresas']:
            nombre_empresa = empresa.get('nombre', 'No disponible')
            empresas_relacionadas.append(nombre_empresa)

    return {
        'ID': id_caso,
        'Name': nombre_sancionado,
        'Type': 'Private',
        "State": state_caso
    }

In [ ]:
def extract_public(caso):
    # Asegurarse de que 'sanciones_servidores' existe y tiene al menos un elemento

    # Extracción del nombre del funcionario público sancionado
    nombre_funcionario = caso.get('nombre_funcionario', 'No disponible')

    # Extracción del ID del caso
    id_caso = caso.get('id_caso', 'No disponible')
    state_caso = caso.get('folder_estado', 'No Disponible')

    return {
        'Name': nombre_funcionario,
        'ID': id_caso,
        "Type": "Public Servant",
        "State": state_caso
    }

In [ ]:
lista_privados = [extract_private(caso) for caso in sanciones['sanciones_privadas']]
lista_publicos = [extract_public(caso) for caso in sanciones['sanciones_servidores']]


In [155]:
import pandas as pd

df = pd.read_csv("sanciones/Listado_Completo_69-B.csv", encoding="latin-1", skiprows=2)

def extract_sat(caso):
  return {
      'Name': caso["Nombre del Contribuyente"],
      "ID": caso["RFC"],
      "Type": "SAT",
      "State": "N/A"
  }

In [156]:
lista_sat = [extract_sat(row) for i,row in df.iterrows()]

In [157]:
lista_sancionados = lista_privados + lista_publicos + lista_sat
json.dump(lista_sancionados, open("sancionados.json", "w"))

In [158]:
json.dump(lista_sancionados, open("output_sancionados.json", 'w'))